## Use case #1: using file inputParams.json (recommended use)

Execute the following cells only the first time running this notebook:

In [1]:
import os
os.chdir("..")

import pandas as pd
pd.set_option('display.precision', 2)

Load the input parameters file in a new instance of the `controller` an request the first `DataFrame` to the specified data source.

In [2]:
from synthDataGen.base import Controller

controller = Controller()
controller.loadMainParams("inputParams.json", "./synthDataGen/settings/")
df = controller.getDataFromSource()
df

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
datetime_utc,,,,,,,,,,,,,,,,
2023-07-20 12:00:00,52.45,80.54,48.15,61.48,67.19,69.81,60.81,45.71,87.96,56.33,62.34,73.41,59.95,45.22,101.99,203.28
2023-07-20 13:00:00,51.54,77.47,48.39,59.97,64.75,66.29,60.33,42.33,85.37,53.87,61.04,72.04,56.68,44.00,100.60,205.59
2023-07-20 14:00:00,51.37,72.49,48.21,59.24,64.20,63.48,59.53,40.98,84.74,52.64,60.67,71.05,52.78,42.40,100.61,201.83
2023-07-20 15:00:00,51.43,68.15,48.77,59.03,64.78,63.00,58.82,40.58,85.39,52.26,60.25,69.23,51.07,42.42,100.61,193.05
2023-07-20 16:00:00,48.37,65.74,48.40,57.86,63.16,63.26,57.29,43.67,85.14,51.12,59.40,66.14,50.90,45.08,99.99,189.72
2023-07-20 17:00:00,46.60,66.02,46.02,54.48,61.25,61.38,58.65,44.19,80.79,49.83,56.41,65.33,51.17,44.02,109.28,193.88
2023-07-20 18:00:00,43.87,65.34,45.04,54.71,62.56,61.24,59.81,48.21,80.50,50.36,56.22,67.26,54.31,44.99,113.83,212.22
2023-07-20 19:00:00,40.96,68.58,43.62,52.92,62.64,61.46,61.15,53.47,81.96,52.22,57.16,69.30,56.00,45.53,115.15,217.59
2023-07-20 20:00:00,40.84,79.22,43.71,53.21,62.25,62.90,62.91,64.16,81.84,51.35,56.67,69.04,56.04,45.64,107.67,214.93


Adjust the source `DataFrame` with the anual adjustments dictionary and resample it to the desired resolution:

In [3]:
from synthDataGen.base import Adjustments

adjustments = Adjustments(controller.inputJSON)
df = adjustments.performAnualAdjustments(df)
df

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
datetime_utc,,,,,,,,,,,,,,,,
2023-07-20 12:00:00,57.70,88.59,52.97,67.63,73.91,76.79,66.89,50.28,96.76,61.96,68.57,80.75,65.95,49.74,112.19,223.61
2023-07-20 13:00:00,56.69,85.22,53.23,65.97,71.23,72.92,66.36,46.56,93.91,59.26,67.14,79.24,62.35,48.40,110.66,226.15
2023-07-20 14:00:00,56.51,79.74,53.03,65.16,70.62,69.83,65.48,45.08,93.21,57.90,66.74,78.16,58.06,46.64,110.67,222.01
2023-07-20 15:00:00,56.57,74.97,53.65,64.93,71.26,69.30,64.70,44.64,93.93,57.49,66.28,76.15,56.18,46.66,110.67,212.36
2023-07-20 16:00:00,53.21,72.31,53.24,63.65,69.48,69.59,63.02,48.04,93.65,56.23,65.34,72.75,55.99,49.59,109.99,208.69
2023-07-20 17:00:00,51.26,72.62,50.62,59.93,67.38,67.52,64.52,48.61,88.87,54.81,62.05,71.86,56.29,48.42,120.21,213.27
2023-07-20 18:00:00,48.26,71.87,49.54,60.18,68.82,67.36,65.79,53.03,88.55,55.40,61.84,73.99,59.74,49.49,125.21,233.44
2023-07-20 19:00:00,45.06,75.44,47.98,58.21,68.90,67.61,67.27,58.82,90.16,57.44,62.88,76.23,61.60,50.08,126.67,239.35
2023-07-20 20:00:00,44.92,87.14,48.08,58.53,68.48,69.19,69.20,70.58,90.02,56.49,62.34,75.94,61.64,50.20,118.44,236.42


In [4]:
df = adjustments.upsample(df)
df

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
2023-07-20 12:00:00,57.70,88.59,52.97,67.63,73.91,76.79,66.89,50.28,96.76,61.96,68.57,80.75,65.95,49.74,112.19,223.61
2023-07-20 12:15:00,57.37,87.99,53.10,67.13,73.04,75.78,66.80,49.11,95.83,61.16,68.10,80.32,65.16,49.47,111.65,224.86
2023-07-20 12:30:00,57.10,87.22,53.19,66.69,72.30,74.80,66.69,48.10,95.05,60.44,67.71,79.92,64.30,49.16,111.21,225.70
2023-07-20 12:45:00,56.87,86.30,53.23,66.30,71.70,73.85,66.54,47.25,94.41,59.80,67.39,79.56,63.36,48.80,110.88,226.13
2023-07-20 13:00:00,56.69,85.22,53.23,65.97,71.23,72.92,66.36,46.56,93.91,59.26,67.14,79.24,62.35,48.40,110.66,226.15
2023-07-20 13:15:00,56.56,83.97,53.18,65.69,70.88,72.02,66.16,46.03,93.54,58.79,66.98,78.96,61.26,47.95,110.55,225.76
2023-07-20 13:30:00,56.48,82.57,53.09,65.46,70.67,71.15,65.92,45.66,93.32,58.42,66.88,78.72,60.09,47.46,110.54,224.97
2023-07-20 13:45:00,56.46,81.12,53.02,65.29,70.59,70.39,65.68,45.37,93.22,58.12,66.82,78.47,58.97,47.00,110.59,223.73
2023-07-20 14:00:00,56.51,79.74,53.03,65.16,70.62,69.83,65.48,45.08,93.21,57.90,66.74,78.16,58.06,46.64,110.67,222.01
2023-07-20 14:15:00,56.62,78.42,53.13,65.08,70.77,69.46,65.32,44.78,93.31,57.76,66.64,77.78,57.34,46.39,110.77,219.82


In [5]:
df = adjustments.downsample(df)
df

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
2023-07-20 12:00:00,56.91,85.37,53.12,66.27,71.79,73.46,66.38,47.30,94.51,59.74,67.45,79.49,62.68,48.50,111.03,225.11
2023-07-20 14:00:00,56.05,75.78,53.46,64.86,70.84,69.49,64.61,45.29,93.78,57.41,66.32,76.19,56.61,47.12,110.36,214.40
2023-07-20 16:00:00,51.33,72.34,51.22,61.05,68.16,68.09,64.23,49.15,90.14,55.24,62.91,72.34,56.72,48.98,118.02,215.02
2023-07-20 18:00:00,45.88,76.40,48.47,58.93,68.87,67.87,67.18,59.23,89.79,56.76,62.52,75.61,61.21,50.03,125.10,237.74
2023-07-20 20:00:00,45.23,83.75,47.08,56.34,67.96,67.13,69.01,68.29,87.15,55.59,61.86,75.32,60.09,48.49,115.87,233.11


Get new samples from the current `DataFrame`:

In [6]:
from synthDataGen.base import Sampling

sampling = Sampling(controller.inputJSON)
df = sampling.getSamples(df)
df.head()

,2023-07-20 12:00:00,2023-07-20 14:00:00,2023-07-20 16:00:00,2023-07-20 18:00:00,2023-07-20 20:00:00
0,21.56,60.52,11.03,103.12,24.37
1,75.48,81.34,61.47,8.95,67.05
2,97.11,105.74,45.73,65.91,95.49
3,119.54,102.18,101.69,100.58,33.85
4,130.39,39.46,108.18,78.63,78.48


# Use case #2: passing parameters


In [7]:
from datetime import datetime

from synthDataGen.base import Controller

controller = Controller()
controller.loadMainParams("inputParams.json", "./synthDataGen/settings/")
df = controller.getDataFromSource(initialYear=2018, initDatetime=datetime(2023, 6, 5, 7, 0), hoursAhead=6)
df

,2018,2019,2020,2021,2022
datetime_utc,,,,,
2023-06-05 06:00:00,67.83,53.13,37.07,81.00,193.53
2023-06-05 07:00:00,69.11,50.74,37.06,81.43,177.57
2023-06-05 08:00:00,69.87,50.44,35.33,85.35,174.79
2023-06-05 09:00:00,69.09,50.50,34.96,85.31,177.76
2023-06-05 10:00:00,68.72,51.26,35.24,85.29,177.64
2023-06-05 11:00:00,69.03,49.11,35.13,85.36,176.71
2023-06-05 12:00:00,69.02,47.22,32.09,83.26,173.96


In [8]:
from synthDataGen.base import Adjustments

adjustments = Adjustments(controller.inputJSON)
df = adjustments.performAnualAdjustments(df, adjustmentsDict={2018: 1.2, 2019: 2.3, 2020: 1.45, 2021: 3, 2022: 8})
df

,2018,2019,2020,2021,2022
datetime_utc,,,,,
2023-06-05 06:00:00,68.64,54.35,37.61,83.43,209.01
2023-06-05 07:00:00,69.94,51.91,37.60,83.87,191.78
2023-06-05 08:00:00,70.71,51.60,35.84,87.91,188.77
2023-06-05 09:00:00,69.92,51.66,35.47,87.87,191.98
2023-06-05 10:00:00,69.54,52.44,35.75,87.85,191.85
2023-06-05 11:00:00,69.86,50.24,35.64,87.92,190.85
2023-06-05 12:00:00,69.85,48.31,32.56,85.76,187.88


In [9]:
df = adjustments.upsample(df, frequency="20T", method="spline", order=3)
df

,2018,2019,2020,2021,2022
2023-06-05 06:00:00,68.64,54.35,37.61,83.43,209.01
2023-06-05 06:20:00,69.25,53.31,37.41,83.74,201.75
2023-06-05 06:40:00,69.70,52.60,36.97,84.39,196.71
2023-06-05 07:00:00,69.94,51.91,37.60,83.87,191.78
2023-06-05 07:20:00,70.24,51.79,36.42,85.63,190.74
2023-06-05 07:40:00,70.36,51.62,36.27,86.21,189.39
2023-06-05 08:00:00,70.71,51.60,35.84,87.91,188.77
2023-06-05 08:20:00,70.36,51.59,36.12,87.20,188.97
2023-06-05 08:40:00,70.28,51.67,36.09,87.61,189.48
2023-06-05 09:00:00,69.92,51.66,35.47,87.87,191.98


In [10]:
df = adjustments.downsample(df, frequency="22.73T", aggregationFunc="mean")
df

,2018,2019,2020,2021,2022
2023-06-05 05:40:57.000,68.64,54.35,37.61,83.43,209.01
2023-06-05 06:03:40.800,69.25,53.31,37.41,83.74,201.75
2023-06-05 06:26:24.600,69.70,52.60,36.97,84.39,196.71
2023-06-05 06:49:08.400,69.94,51.91,37.60,83.87,191.78
2023-06-05 07:11:52.200,70.24,51.79,36.42,85.63,190.74
2023-06-05 07:34:36.000,70.36,51.62,36.27,86.21,189.39
2023-06-05 07:57:19.800,70.53,51.60,35.98,87.56,188.87
2023-06-05 08:20:03.600,70.28,51.67,36.09,87.61,189.48
2023-06-05 08:42:47.400,69.92,51.66,35.47,87.87,191.98
2023-06-05 09:05:31.200,70.02,51.84,36.01,88.18,190.97


In [11]:
from synthDataGen.base import Sampling

sampling = Sampling(controller.inputJSON)
df = sampling.getSamples(df, 1000, "truncnorm")
df.head()

,2023-06-05 05:40:57.000,2023-06-05 06:03:40.800,2023-06-05 06:26:24.600,2023-06-05 06:49:08.400,2023-06-05 07:11:52.200,2023-06-05 07:34:36.000,2023-06-05 07:57:19.800,2023-06-05 08:20:03.600,2023-06-05 08:42:47.400,2023-06-05 09:05:31.200,2023-06-05 09:28:15.000,2023-06-05 09:50:58.800,2023-06-05 10:13:42.600,2023-06-05 10:36:26.400,2023-06-05 10:59:10.200,2023-06-05 11:21:54.000,2023-06-05 11:44:37.800
0,89.52,133.31,92.54,112.58,18.94,176.15,53.76,87.78,107.17,115.06,60.88,13.12,108.71,66.82,45.01,98.96,107.37
1,117.24,83.50,52.35,123.09,57.38,122.74,48.54,45.29,136.00,79.81,67.83,35.63,60.34,57.02,175.50,44.26,164.89
2,88.96,9.56,83.74,40.44,21.61,13.19,130.80,37.85,27.32,99.28,86.06,111.17,185.14,65.17,123.06,50.96,151.45
3,176.29,87.38,33.23,12.97,78.85,139.45,103.56,94.13,20.49,161.78,119.62,81.04,52.85,112.15,100.17,37.30,22.26
4,45.26,112.36,110.15,32.75,72.53,65.84,141.43,27.92,139.92,122.55,73.27,150.20,125.34,196.29,27.03,146.91,7.89
